##MNIST Digit Classifer

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

import torchvision
import torchvision.transforms.v2 as transforms
import torchvision.transforms.functional as F
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
train_set = torchvision.datasets.MNIST("./data/", train=True, download=True)
valid_set = torchvision.datasets.MNIST("./data/", train=False, download=True)

100.0%
100.0%
100.0%
100.0%


In [ ]:
#For converting image to tensor
trans = transforms.Compose([transforms.ToImage(), transforms.ToDtype(torch.float32, scale=True)])

In [9]:
train_set.transform = trans
valid_set.transform = trans

In [10]:
#Setting batch size
batch_size = 32

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size)

In [11]:
input_size = 1 * 28 * 28
n_classes = 10

In [13]:
layers = [
    nn.Flatten(),
    nn.Linear(input_size, 512),  # Input
    nn.ReLU(),  # Activation for input
    nn.Linear(512, 512),  # Hidden
    nn.ReLU(),  # Activation for hidden
    nn.Linear(512, n_classes)  # Output
]

In [26]:
model = nn.Sequential(*layers)

In [ ]:
# model = torch.compile(model)

In [16]:
loss_function = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters())

In [17]:
train_N = len(train_loader.dataset)
valid_N = len(valid_loader.dataset)

In [18]:
def get_batch_accuracy(output, y, N):
    pred = output.argmax(dim=1, keepdim=True)
    correct = pred.eq(y.view_as(pred)).sum().item()
    return correct / N

In [24]:
def train():
    loss = 0
    accuracy = 0

    model.train()
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        output = model(x)
        optimizer.zero_grad()
        batch_loss = loss_function(output, y)
        batch_loss.backward()
        optimizer.step()

        loss += batch_loss.item()
        accuracy += get_batch_accuracy(output, y, train_N)
    print('Train - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

In [20]:
def validate():
    loss = 0
    accuracy = 0

    model.eval()
    with torch.no_grad():
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            output = model(x)

            loss += loss_function(output, y).item()
            accuracy += get_batch_accuracy(output, y, valid_N)
    print('Valid - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

In [27]:
epochs = 5

for epoch in range(epochs):
    print('Epoch: {}'.format(epoch))
    train()
    validate()

Epoch: 0
Train - Loss: 377.9450 Accuracy: 0.9376
Valid - Loss: 37.5583 Accuracy: 0.9608
Epoch: 1
Train - Loss: 156.9905 Accuracy: 0.9744
Valid - Loss: 25.2254 Accuracy: 0.9758
Epoch: 2
Train - Loss: 106.5283 Accuracy: 0.9822
Valid - Loss: 25.1043 Accuracy: 0.9762
Epoch: 3


KeyboardInterrupt: 

In [28]:
x_0, y_0 = train_set[0]

In [30]:
prediction = model(trans(x_0))
prediction

tensor([[-22.2152,  -1.5843, -14.3528,  10.8978, -25.2989,  15.5638, -21.3509,
         -12.2016,  -9.6895,  -6.4808]], grad_fn=<AddmmBackward0>)

In [31]:
prediction.argmax(dim=1, keepdim=True)

tensor([[5]])

In [32]:
y_0

5